# Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
import os
from math import pi
%matplotlib inline

In [ ]:
#Run this to reset the results
results= pd.DataFrame(columns = ['LC Name', 'Temperature', 'Applied Voltage','Rise Time','Decay Time','Total Response Time'])

In [ ]:
#Loading the data
path="/home/avinash/Desktop/Projects/Liquid_Crystal_Analysis/data/Batch2" #Enter Loaction WHere CSV data is kept

#temp= str(input("Enter Temperature: ")) #If you have a fixed value of temp, then comment this line and uncomment the next line after giving value
#temp =str(30)
name_of_LC="Pure LC"# Change this to show type of LC
thickness= 7.01E-6 #in Metres
area=77E-6 #in Metres^2
empty_c=99.6E-12 #in Farad
wavelength=633E-9 #in metre
if not os.path.exists(path + "/Analysis") :
    os.mkdir (path + "/Analysis")
output_path=path+"/Analysis"

# Analysis of Each Voltage Case

In [ ]:
#Input Voltage
try :
    temp=str(input("Enter Temperature: "))
    voltage=str(input("Enter Voltage: "))
    input_path=path+"/"+temp+"_"+voltage+"_2.csv"
    df=pd.read_csv(input_path,skiprows=1)
    #Normalizing the Volt to represent transmittance
    df.Volt=(df.Volt-df.Volt.min())/(df.Volt.max()-df.Volt.min())
except :
  print ("Incorrect CSV directory or filename")
  exit(1)
else:
    print("File Exists. Start Analysis")

In [ ]:
plt.figure(figsize=(12, 8), dpi=100, linewidth=0.1)
plt.xlabel("Time (in seconds)")
plt.ylabel("Transmittance")

#Have set the window size as a constant = 33. If the number of intersections on
#the 0.9 line or 0.1 line are more than 6, then icncrease window size
window_size =33
smooth=savgol_filter(df.Volt,window_size,2) #Creating a smoothened Curve us SavGol filter with window size 25 

plt.plot(df.second,smooth) #Printing the smooth curve
plt.plot(df.second,df.Volt)#Printing the actual curve

plt.plot(df.second,0.9*np.ones(df.second.shape)) #Printing horizontal line at 0.9
plt.plot(df.second,0.1*np.ones(df.second.shape)) #Printing horizontal line at 0.1

#Finding position where the curve closses the 0.9 transmittance
idx = np.argwhere(np.diff(np.sign(smooth - 0.9))).flatten()
if(len(idx)!=6):
    print("Error in top. Number of points greater than 6. Increase window size and retry")
    exit(1)
idx=idx[1:5] #Eliminating First and last points
plt.plot(df.second[idx], 0.9*np.ones(idx.size), 'ro') #Plotting Points of intersection
#print(len(idx))
idx_top=idx
for i in range(len(idx)):
    plt.annotate("T90="+str(round(df.second[idx[i]],4)),(df.second[idx[i]], 0.9),xytext =(df.second[idx[i]]+0.0005-max(0,0.006*pow(-1,i+1)), 0.91))


#Finding position where the curve closses the 0.1 transmittance
idx = np.argwhere(np.diff(np.sign(smooth - 0.1))).flatten()
if(len(idx)!=6):
    print("Error in bot. Number of points greater than 6. Increase window size and retry")
    exit(1)
idx=idx[1:5] #Eliminating First and last points
plt.plot(df.second[idx], 0.1*np.ones(idx.size), 'bo') #Plotting Points of intersection
#print(len(idx))
idx_bot=idx
for i in range(len(idx)):
    plt.annotate("T10="+str(round(df.second[idx[i]],4)),(df.second[idx[i]], 0.1),xytext =(df.second[idx[i]]+0.0005-max(0,0.006*pow(-1,i)), 0.11-max(0,0.05*pow(-1,i))))

#Calculating RiseTime and DecayTime (average of 2 sets)
rise_time=((df.second[idx_bot[0]]-df.second[idx_top[0]])+(df.second[idx_bot[2]]-df.second[idx_top[2]]))/2
decay_time=((df.second[idx_top[1]]-df.second[idx_bot[1]])+(df.second[idx_top[3]]-df.second[idx_bot[3]]))/2
response_time=rise_time+decay_time
#Printing the times
print("Rise Time = ",rise_time,"seconds")
print("Decay Time = ",decay_time,"seconds")
print("Response Time = ",response_time,"seconds")
plt.annotate("Rise Time = "+str(round(rise_time,4)),xy=(0.019,1))
plt.annotate("Decay Time="+str(round(decay_time,4)),xy=(0.019,0.97))
plt.annotate("Resp. Time="+str(round(response_time,4)),xy=(0.019,0.94))
plt.annotate(name_of_LC,xy=(-0.027,1))
plt.annotate("Appl. Volt. = "+voltage+"V",xy=(-0.027,0.97))
plt.annotate("Temp = "+temp+"C",xy=(-0.027,0.94))

plt.savefig(output_path+"/"+temp+"_"+voltage+".jpg", dpi=300,bbox_inches='tight')
plt.show()


In [ ]:
#If you are happy with the above results add it to the list. Execute this command to do that.

results=results.append({'LC Name':name_of_LC, 'Temperature':temp, 'Applied Voltage':voltage,'Rise Time':rise_time,'Decay Time':decay_time,'Total Response Time': response_time},ignore_index=True)
results.tail()

In [ ]:
#Plotting the response time vs Applied Voltage
plt.figure(1,figsize=(12, 8), dpi=100, linewidth=0.1)
plt.ylabel("Time (in seconds)")
plt.xlabel("Applied Voltage (in Volts)")
plt.plot(results['Applied Voltage'],results['Total Response Time'],'-o')
plt.plot(results['Applied Voltage'],results['Rise Time'],'-o')
plt.plot(results['Applied Voltage'],results['Decay Time'],'-o')
plt.legend(['Total Response Time','Rise Time','Decay Time'])
plt.savefig(output_path+"/"+name_of_LC+"_"+temp+".jpg", dpi=300,bbox_inches='tight')
plt.show()


In [ ]:
#If you are happy with the results, you can write it to a csv file.
results.to_csv(output_path+"/"+"RespTimevsAppVolt.csv", index=False)

# Analysing properties of the Liquid Crystal

In [ ]:
#Setting up dataframe for recording derived properties 
properties=pd.DataFrame(columns = ['Property', 'Value', 'Units'])

In [ ]:


try :
    results=pd.read_csv(output_path+"/"+"RespTimevsAppVolt.csv")
except :
  print ("Incorrect CSV directory or filename")
  exit(1)
else:
    print("File Exists. Further calculations allowed")


In [ ]:
# Rotational_Viscosity/Combined_Elastic_Constant Analysis
prop="Gamma/K"
value=results['Decay Time'].mean()*pi**2/thickness**2
units="s/m^2"
print("Rotational_Viscosity/Combined_Elastic_Constant (gamma/K) = "+str(value)+" "+units)


In [ ]:
#If the above Property is fine, append the property to list of properties
properties=properties.append({'Property':prop, 'Value':value, 'Units':units},ignore_index=True)
properties.tail()

In [ ]:
# If all the properties are fine, write out the properties to a csv file
properties.to_csv(output_path+"/"+"Derived_Properties.csv", index=False)